In [1]:
#импорт необходимых библиотек
import pandas as pd
import requests
import csv

In [2]:
from bs4 import BeautifulSoup

In [3]:
import random
import time

In [4]:
# импорт библиотеки создания базы данных
import sqlite3

In [5]:
# подключение к базе данных
connection = sqlite3.connect('apartments.db')
# создание объекта подключения
cursor = connection.cursor()
print("База данных успешно подключена к SQLite")

База данных успешно подключена к SQLite


In [6]:
# вывод наименований созданных таблиц
cursor.execute('SELECT name from sqlite_master where type= "table"')
print(cursor.fetchall())

[('as_house_type',), ('as_info',), ('as_history',), ('as_price_val',), ('as_seller_type',), ('as_seller_info',)]


In [7]:
cursor.execute("PRAGMA table_info('as_info')")
as_info = [name[1] for name in cursor.fetchall()]
print(as_info)

['marker_id', 'href', 'title', 'house_id', 'address', 'seller_id', 'description', 'priority', 'params', 'map_lat', 'map_lon']


In [8]:
#connection.close()

In [9]:
from datetime import date
current_date = date.today()
print(current_date)

2022-03-17


In [10]:
def get_html(url):
    r = requests.get(url)
    
    return r.text

In [11]:
def get_total_pages(html):
    soup = BeautifulSoup(html, 'html.parser')
    pages = soup.find('div', {'class':'pagination-root-Ntd_O'}).find_all('span', {'class':'pagination-item-JJq_j'})
    total_pages = pages[-2].text
    
    return int(total_pages)

In [12]:
def write_db(data):
    
    val_name = data['price_val']
    cursor.execute("SELECT val_id FROM as_price_val WHERE val_name = ?;", (val_name,))
    val_id = cursor.fetchall()[0][0]
    
    cursor.execute("SELECT * FROM as_history WHERE marker_id = ? AND date = ?;", (data['marker_id'], current_date))
    marker_id_date = cursor.fetchall()
     
    try:
        marker_id_date[0]
    except:
        cursor.execute("INSERT INTO as_history (marker_id, price, val_id, date) VALUES(?, ?, ?, ?);", \
                   (data['marker_id'], data['price'], val_id, current_date))
        connection.commit()
    
    type_name = data['seller']
    seller_name = data['seller_name']  
    
    cursor.execute("SELECT type_id FROM as_seller_type WHERE type_name = ?;", (type_name,))
    type_id = cursor.fetchall()[0][0]
    
    if type_id == 2:
        seller_name = 'Собственник'
    
    try:
        cursor.execute("INSERT INTO as_seller_info (type_id, seller_name) VALUES(?, ?);", \
                       (type_id, seller_name))
        connection.commit()
    except:
        pass
    
    try:
        cursor.execute("SELECT seller_id FROM as_seller_info WHERE seller_name = ?;", (seller_name,))
        seller_id = cursor.fetchall()[0][0]
    except:
        pass    
    
    try:
        cursor.execute("INSERT INTO as_info (marker_id, href, title, address, seller_id, description, priority) \
                   VALUES (?, ?, ?, ?, ?, ?, ?);", (data['marker_id'], data['href'], data['title'], \
                                              data['address'], seller_id, data['description'], data['priority']))
        connection.commit()
    except:
        pass

In [13]:
def get_pages_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    ads = soup.find_all('div', {'data-marker': 'item'})
    for ad in ads:
        try:
            marker_id = ad['id'][1:]
        except:
            marker_id = None    
        try:
            href = 'https://www.avito.ru' + ad.find('a')['href']
        except:
            href = None
        try:
            title = ad.find('div', {'class' : 'iva-item-titleStep-pdebR'}).text    
        except:
            title = None
        try:
            price = ''.join(ad.find('span', {'class' : 'price-price-JP7qe'}).text.split()[:-1])
        except:    
            price = None
        try:
            price_val = ad.find('span', {'class' : 'price-price-JP7qe'}).text.replace(r'₽','руб.').split()[-1]   
        except:    
            price_val = None
        try:
            address = ad.find('div', {'class' : 'geo-root-zPwRk iva-item-geo-_Owyg'}).text
        except:
            address = None
        try:
            seller_name =  ad.find('div', {'class' : 'style-root-uufhX'}).text
        except:
            seller_name = None
        try:
            ad.find('div', {'data-marker' : 'item-line'}).text
            seller = 'Агентство'
        except:
            seller = 'Собственник'       
        try:    
            description = ad.find('div', {'class' : 'iva-item-descriptionStep-C0ty1'}).text
        except:    
            description = None
        try:
            prioritys = ad.find_all('div', {'class' : 'SnippetBadge-root-ZnUY6'})
            priority = [priority.text for priority in prioritys]
            if len(priority) == 1:
                priority = priority[0]
            else:
                priority = ', '.join(priority)
        except:    
            priority = None    
        
        
        data = {'marker_id':marker_id,
                'href' : href,
                'title' : title,
                'price' : price,
                'price_val' : price_val,
                'address' : address, 
                'seller_name' : seller_name, 
                'seller' : seller,
                'description' : description,
                'current_date' : current_date,
                'priority' : priority}
        
        write_db(data)

In [14]:
def main():
    url = 'https://www.avito.ru/zelenodolsk/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1'
    base_url = 'https://www.avito.ru/zelenodolsk/kvartiry/prodam-ASgBAgICAUSSA8YQ?cd=1&p={}'
    for p in range(1, get_total_pages(get_html(url)) + 1): #get_total_pages(get_html(url)) + 1
        url_gen = base_url.format(p)
        html = get_html(url_gen)
        get_pages_data(html)
        pause = round(random.uniform(10, 30))
        time.sleep(pause)

In [15]:
if __name__ == '__main__':
    main()

In [16]:
connection.close()